# Cleaning the dataset, gathering geometry 

### This notebook is to clean the csv containing property listings from Daft.ie and obtain their coordinates


In [91]:
import pandas as pd
import re
import json
import geopandas as gpd

In [2]:
from datetime import date

today = date.today()
year = today.year
month = today.month
day = today.day

# df = pd.read_csv(f"./scrape_output/daft_listings_{year}{month}{day}.csv")
df = pd.read_csv(f"./scrape_output/daft_listings_20221228.csv")

In [3]:
# remove listings for which we don't have a price
df.drop(df[df['price'] == "Price on Application "].index, inplace = True)
df = df.dropna()

In [4]:
df

,Unnamed: 0,price,address,bedrooms,bathrooms,floor_area
20,20,"€470,000","Apartment 17, Amber, The Grange, Stillorgan, C...",2 Bed,2 Bath,74 m²
21,21,"€435,000","Apartment 1, House 5, Linden Square, Blackrock...",2 Bed,2 Bath,79 m²
22,22,"€1,995,000","Karroc, 14 Myra Manor, Malahide, Co. Dublin",5 Bed,4 Bath,481 m²
23,23,"€485,000","11 Reuben Avenue, Rialto, Rialto, Dublin 8",2 Bed,1 Bath,90 m²
24,24,"€1,975,000","Chipton, 25 Ailesbury Park, Ballsbridge, Dublin 4",5 Bed,2 Bath,285 m²
...,...,...,...,...,...,...
3107,3107,"€250,000","16 Healy Street, Dublin 1",2 Bed,1 Bath,60 m²
3109,3109,"€350,000","18 The Glade, Hunters Run, Clonee, Dublin 15",3 Bed,3 Bath,110 m²
3110,3110,"€340,000","6 Rathmore Avenue, Tyrellstown, Tyrrelstown, D...",3 Bed,3 Bath,100 m²
3111,3111,"€800,000","14 Lambay Court, Malahide, Co. Dublin",4 Bed,2 Bath,140 m²


In [5]:
df["bedrooms"] = df["bedrooms"].str.replace(" Bed", "")
df["bedrooms"] = df["bedrooms"].astype(int)

In [6]:
df["bathrooms"] = df["bathrooms"].str.replace(" Bath", "")
df["bathrooms"] = df["bathrooms"].astype(int)

In [7]:
df["price"] = df["price"].str.replace("€", "")
df["price"] = df["price"].str.replace(",", "")

# to remove of any entries that start with AMV (advised minimum value)
df["price"] = df["price"].str.replace(r"^AMV:\s", "")

df["price"] = df["price"].astype(int)

/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_54582/77379337.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df["price"] = df["price"].str.replace(r"^AMV:\s", "")


In [8]:
df["floor_area"] = df["floor_area"].str.replace("m²", "")

In [9]:
df[df["floor_area"].str.contains("ac")==True]
#assuming this one is wrong - seems like the value is in metres sq but is labelled incorrectly (from inspecting df / csv)

,Unnamed: 0,price,address,bedrooms,bathrooms,floor_area
2896,2896,495000,"96 Beech Grove Cottages, Bray Road, Loughlinst...",4,2,748.73 ac


In [10]:
#dropping that row
df = df.drop([2896])

In [11]:
df["floor_area"] = df["floor_area"].astype(float)

In [12]:
df["price_sqm"] = round(df["price"]/df["floor_area"])

In [74]:
df.to_csv(f"./cleaned_output/clean_output_{year}{month}{day}.csv", index=False)

In [72]:
df

,Unnamed: 0,price,address,bedrooms,bathrooms,floor_area,price_sqm
20,20,470000,"Apartment 17, Amber, The Grange, Stillorgan, C...",2,2,74.0,6351.0
21,21,435000,"Apartment 1, House 5, Linden Square, Blackrock...",2,2,79.0,5506.0
22,22,1995000,"Karroc, 14 Myra Manor, Malahide, Co. Dublin",5,4,481.0,4148.0
23,23,485000,"11 Reuben Avenue, Rialto, Rialto, Dublin 8",2,1,90.0,5389.0
24,24,1975000,"Chipton, 25 Ailesbury Park, Ballsbridge, Dublin 4",5,2,285.0,6930.0
...,...,...,...,...,...,...,...
3107,3107,250000,"16 Healy Street, Dublin 1",2,1,60.0,4167.0
3109,3109,350000,"18 The Glade, Hunters Run, Clonee, Dublin 15",3,3,110.0,3182.0
3110,3110,340000,"6 Rathmore Avenue, Tyrellstown, Tyrrelstown, D...",3,3,100.0,3400.0
3111,3111,800000,"14 Lambay Court, Malahide, Co. Dublin",4,2,140.0,5714.0


# Get the lat lon for each listing

In [15]:
! pip install python-dotenv

In [16]:
import os
import requests
import time


from dotenv import load_dotenv
load_dotenv()

True

In [17]:
api_key = os.getenv('key')

# api_key = "API KEY HERE"

In [18]:
# we're going to work with our df as a dict 

addresses = df["address"].tolist()


In [58]:
coordinates_df = []
#looping through the placeNames list
for place in addresses:
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    keys = {'address': place, 'key': api_key}
    r = requests.get(url,params=keys)
    result_dic = r.json()
    lat = result_dic['results'][0]['geometry']['location']['lat']
    long = result_dic['results'][0]['geometry']['location']['lng']
    coords = []
    coords.append(long)
    coords.append(lat)
    mygeometry = {'place': place, 'geometry.type': 'Point','geometry.coordinates':coords, 'latitude': lat, 'longitude': long}
    coordinates_df.append(mygeometry)
    print(mygeometry)
    time.sleep(0.5)

{'place': 'Apartment 17, Amber, The Grange, Stillorgan, Co. Dublin', 'geometry.type': 'Point', 'geometry.coordinates': [-6.1928515, 53.280368], 'latitude': 53.280368, 'longitude': -6.1928515}
{'place': 'Apartment 1, House 5, Linden Square, Blackrock, Co. Dublin', 'geometry.type': 'Point', 'geometry.coordinates': [-6.19043, 53.29279], 'latitude': 53.29279, 'longitude': -6.19043}
{'place': 'Karroc, 14 Myra Manor, Malahide, Co. Dublin', 'geometry.type': 'Point', 'geometry.coordinates': [-6.173941399999999, 53.4327365], 'latitude': 53.4327365, 'longitude': -6.173941399999999}
{'place': '11 Reuben Avenue, Rialto, Rialto, Dublin 8', 'geometry.type': 'Point', 'geometry.coordinates': [-6.2922444, 53.3351852], 'latitude': 53.3351852, 'longitude': -6.2922444}
{'place': 'Chipton, 25 Ailesbury Park, Ballsbridge, Dublin 4', 'geometry.type': 'Point', 'geometry.coordinates': [-6.211598, 53.3203349], 'latitude': 53.3203349, 'longitude': -6.211598}
{'place': '4 Linnetfields Park, Clonee, Dublin 15', 'g

In [66]:
len(coordinates_df)

# make sure it is the same as the number of rows in the df. (It is, yay!)

3078

In [67]:
with open("listings_coordinates.json", "w") as outfile:
    json.dump(coordinates_df, outfile)

In [61]:
! ls

Cleaning.ipynb            geo-data.js               map.html
Scrape.ipynb              joining.ipynb             scrape_output
analysis_output           listings_geometry.csv     spatial_layers
backups                   listings_geometry.geojson
df_to_maps.ipynb          listings_geometry.json


In [82]:
geometry_df = pd.DataFrame.from_dict(coordinates_df)

geometry_df["address"] = geometry_df["place"]

del geometry_df["place"]

geometry_df

,geometry.type,geometry.coordinates,latitude,longitude,address
0,Point,"[-6.1928515, 53.280368]",53.280368,-6.192851,"Apartment 17, Amber, The Grange, Stillorgan, C..."
1,Point,"[-6.19043, 53.29279]",53.292790,-6.190430,"Apartment 1, House 5, Linden Square, Blackrock..."
2,Point,"[-6.173941399999999, 53.4327365]",53.432736,-6.173941,"Karroc, 14 Myra Manor, Malahide, Co. Dublin"
3,Point,"[-6.2922444, 53.3351852]",53.335185,-6.292244,"11 Reuben Avenue, Rialto, Rialto, Dublin 8"
4,Point,"[-6.211598, 53.3203349]",53.320335,-6.211598,"Chipton, 25 Ailesbury Park, Ballsbridge, Dublin 4"
...,...,...,...,...,...
3073,Point,"[-6.252406800000001, 53.3563434]",53.356343,-6.252407,"16 Healy Street, Dublin 1"
3074,Point,"[-6.4212637, 53.40448180000001]",53.404482,-6.421264,"18 The Glade, Hunters Run, Clonee, Dublin 15"
3075,Point,"[-6.389942, 53.419216]",53.419216,-6.389942,"6 Rathmore Avenue, Tyrellstown, Tyrrelstown, D..."
3076,Point,"[-6.135106899999999, 53.4463976]",53.446398,-6.135107,"14 Lambay Court, Malahide, Co. Dublin"


In [136]:
# joined = pd.merge(df, geometry_df, left_on='address', right_on='address')

joined = df.merge(geometry_df, how='left', on='address')

joined

### look into why the join is slightly different here

,Unnamed: 0,price,address,bedrooms,bathrooms,floor_area,price_sqm,geometry.type,geometry.coordinates,latitude,longitude
0,20,470000,"Apartment 17, Amber, The Grange, Stillorgan, C...",2,2,74.0,6351.0,Point,"[-6.1928515, 53.280368]",53.280368,-6.192851
1,21,435000,"Apartment 1, House 5, Linden Square, Blackrock...",2,2,79.0,5506.0,Point,"[-6.19043, 53.29279]",53.292790,-6.190430
2,22,1995000,"Karroc, 14 Myra Manor, Malahide, Co. Dublin",5,4,481.0,4148.0,Point,"[-6.173941399999999, 53.4327365]",53.432736,-6.173941
3,23,485000,"11 Reuben Avenue, Rialto, Rialto, Dublin 8",2,1,90.0,5389.0,Point,"[-6.2922444, 53.3351852]",53.335185,-6.292244
4,24,1975000,"Chipton, 25 Ailesbury Park, Ballsbridge, Dublin 4",5,2,285.0,6930.0,Point,"[-6.211598, 53.3203349]",53.320335,-6.211598
...,...,...,...,...,...,...,...,...,...,...,...
3105,3107,250000,"16 Healy Street, Dublin 1",2,1,60.0,4167.0,Point,"[-6.252406800000001, 53.3563434]",53.356343,-6.252407
3106,3109,350000,"18 The Glade, Hunters Run, Clonee, Dublin 15",3,3,110.0,3182.0,Point,"[-6.4212637, 53.40448180000001]",53.404482,-6.421264
3107,3110,340000,"6 Rathmore Avenue, Tyrellstown, Tyrrelstown, D...",3,3,100.0,3400.0,Point,"[-6.389942, 53.419216]",53.419216,-6.389942
3108,3111,800000,"14 Lambay Court, Malahide, Co. Dublin",4,2,140.0,5714.0,Point,"[-6.135106899999999, 53.4463976]",53.446398,-6.135107


In [90]:
joined

,Unnamed: 0,price,address,bedrooms,bathrooms,floor_area,price_sqm,geometry.type,geometry.coordinates,latitude,longitude
0,20,470000,"Apartment 17, Amber, The Grange, Stillorgan, C...",2,2,74.0,6351.0,Point,"[-6.1928515, 53.280368]",53.280368,-6.192851
1,21,435000,"Apartment 1, House 5, Linden Square, Blackrock...",2,2,79.0,5506.0,Point,"[-6.19043, 53.29279]",53.292790,-6.190430
2,22,1995000,"Karroc, 14 Myra Manor, Malahide, Co. Dublin",5,4,481.0,4148.0,Point,"[-6.173941399999999, 53.4327365]",53.432736,-6.173941
3,23,485000,"11 Reuben Avenue, Rialto, Rialto, Dublin 8",2,1,90.0,5389.0,Point,"[-6.2922444, 53.3351852]",53.335185,-6.292244
4,24,1975000,"Chipton, 25 Ailesbury Park, Ballsbridge, Dublin 4",5,2,285.0,6930.0,Point,"[-6.211598, 53.3203349]",53.320335,-6.211598
...,...,...,...,...,...,...,...,...,...,...,...
3105,3107,250000,"16 Healy Street, Dublin 1",2,1,60.0,4167.0,Point,"[-6.252406800000001, 53.3563434]",53.356343,-6.252407
3106,3109,350000,"18 The Glade, Hunters Run, Clonee, Dublin 15",3,3,110.0,3182.0,Point,"[-6.4212637, 53.40448180000001]",53.404482,-6.421264
3107,3110,340000,"6 Rathmore Avenue, Tyrellstown, Tyrrelstown, D...",3,3,100.0,3400.0,Point,"[-6.389942, 53.419216]",53.419216,-6.389942
3108,3111,800000,"14 Lambay Court, Malahide, Co. Dublin",4,2,140.0,5714.0,Point,"[-6.135106899999999, 53.4463976]",53.446398,-6.135107


In [85]:
joined.to_csv("listings_coordinates.csv", index=False)

THEN Used the Intersection tool in QGIS to join listings_coordinates.csv to the KML file --> output saved as inteersection.csv for next step